In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [10]:
import tweepy
import csv
import pandas as pd
import re

In [3]:
# getting the tweets from the twitter API
def twitter_query(query_name):
    # add remotely the consumer keys and tokens
    %run ./keys.ipynb
    # connecting to the API
    auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    # initializing tables
    num_of_tweets = 1000
    tweets = []
    likes = []
    time = []
    # getting the result of the query
    for i in tweepy.Cursor(api.search, q=query_name, Since='2021-01-01', lang = 'en', tweet_mode="extended").items(num_of_tweets):
        tweets.append(i.full_text)
        likes.append(i.favorite_count) 
        time.append(i.created_at)
    return tweets, likes, time


In [4]:
# convert the list of tweets into a csv
def tweets_csv(query_name, csv_name):
    # getting the filled lists of results
    tweets, likes, time = twitter_query(query_name)
    # transforming into a DataFrame
    df = pd.DataFrame({'tweets':tweets, 'likes':likes, 'time':time})
    # transforming into a csv
    df.to_csv(csv_name, header=True, index=False, index_label=None, encoding="UTF8")

In [14]:
# cleaning the tweets data
def tweet_cleaning(csv_name):
    df = pd.read_csv(csv_name)
    df = df[~df.tweets.str.contains("RT")]
    df = df.reset_index(drop = True)
    tweets = df['tweets']
    
    for i in range(len(tweets)):
        clean_tweet = re.sub("https://[A-Za-z0-9./_]+", " ", tweets[i]).strip()
        clean_tweet = re.sub("@[A-Za-z0-9_]+"," ", clean_tweet)
        clean_tweet = re.sub("#[A-Za-z0-9_]+"," ", clean_tweet)
        clean_tweet = clean_tweet.lower()
        whitelist = set("'abcdefghijklmnopqrstuvwxyz ")
        clean_tweet = re.sub(":", " ", clean_tweet)
        clean_tweet = ''.join(filter(whitelist.__contains__, clean_tweet))
        tweets[i] = clean_tweet
    return df
